In [3]:
import pandas as pd
import time
import numpy as np
np.seterr(invalid='ignore')
import nbimporter
from PPP import PPP

prevGames = ['Game1']
Opp = 'Nuggets'
Team = 'Heat'
series = 'Finals'

In [2]:
players = ['adebayo', 'butler', 'highsmith', 'love', 'lowry', 'martin', 'robinson', 'strus', 'vincent', 'zeller', 'team', 'opp']

In [35]:
def load_data(players):
    
    # Establish dictionaries for new game data and overall series/season data
    new_player_data = {}
    ovr_player_data = {}
    ovr_series_player_data = {}
    team_opp = ['team', 'opp']
    
    for game in prevGames: # Loop over all games in series/season
        
        for player in players: # Loop over all players on roster
            
            if player == 'team': # Add all player data together to create team file for game

                try: # to load a team file for the game (helpful if already added team file, and/or rerunning for Opp file)
                    newplayerFile = (f'Team/{game + Opp}Team.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the existing new team file to the new_player_data dictionary
                except(FileNotFoundError):
                    # Load an empty df
                    newplayerFile = ('Team/empty.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.set_index('Shot Type')

                    for key, df in new_player_data.items(): # Loop over the new_player_data and add together all player dataframes
                        if key not in team_opp: # Exclude the team and opponent dataframes
                            newplayer = newplayer.add(df)

                    new_player_data['team'] = newplayer # Add the created new team file to new_player_data dictionary
                    newplayer.to_csv(f'../{Opp}/Opp/{game + Team}Opp.csv') # Save team file to opponent folder for their opponent stats
                    newplayer.to_csv(f'Team/{game + Opp}Team.csv') # Save team file to team folder
                    
                # Load overall data for the team
                try:
                    ovrplayerFile = (f'!Data/Team.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add the overall team data to the ovr_player_data dictionary
                except(FileNotFoundError):
                    ovrplayer = newplayer
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/{player.title()}.csv')
                
                # Load overall series data for the player
                try: 
                    ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
                # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
                except(FileNotFoundError):
                    ovr_seriesplayer = newplayer
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
                    continue
                    
                if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                    if game == 'Game1':
                        continue

                    ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
                else:
                    print(f"{player.title()}'s columns do not match")  
                continue

            if player == 'opp': # Load the opponent file and add the new game data to the overall opponent file if necessary

                try: # to load the new opponent file (if running this teams data first, there will be no opponent file!!)
                    newplayerFile = (f'Opp/{game + Opp}Opp.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the new opponent file to the new_player_data dictionary
                except(FileNotFoundError):
                    print(f'No {player.title()} file found.')
                    continue
                    
                # Load overall data for the opponent
                try:
                    ovrplayerFile = (f'!Data/Opp.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add the overall opponent file to the ovr_player_data dictionary
                except(FileNotFoundError):
                    if newplayer:
                        ovrplayer = newplayer
                        ovr_player_data[player] = ovrplayer
                        ovrplayer.to_csv(f'!Data/{player.title()}.csv')
                    else:
                        print(f'No overall Opponent file found.')
                        continue
                        
                # Load overall series data for the player
                try: 
                    ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
                # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
                except(FileNotFoundError):
                    ovr_seriesplayer = newplayer
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
                    continue

                if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                    if game == 'Game1':
                        continue

                    ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
                else:
                    print(f"{player.title()}'s columns do not match")  
                continue
                
            ### Loading/Adding the player files (first part of the loop)###
            try:
                newplayerFile = (f'{player.title()}/{game + Opp + player.title()}.csv') # Search the team directory for the player, then concat the game/opp/player to find csv for game
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.dropna()
                newplayer = newplayer.set_index('Shot Type')
                newplayer = newplayer.astype('int')
                new_player_data[player] = newplayer # Add the new player data to the new_player_data dictionary
            except(FileNotFoundError):
                print(f'No {player.title()} file found for {game}.')
                try: # to load an overall file for the player
                    ovrplayerFile = (f'!Data/{player.title()}.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
                    
                try: # to load an overall player file for the series
                    ovr_seriesplayerFile = (f'!Data/{player.title()}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_series_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No {series} series data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
                    
            # Load overall data for the player
            try: 
                ovrplayerFile = (f'!Data/{player.title()}.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall data for the player, set the new player data as the overall and continue to the next player    
            except(FileNotFoundError):
                ovrplayer = newplayer
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv')
              
            # Load overall series data for the player
            try: 
                ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
                ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                ovr_seriesplayer = ovr_seriesplayer.dropna()
                ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                ovr_seriesplayer = ovr_seriesplayer.astype('int')
                ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
            except(FileNotFoundError):
                ovr_seriesplayer = newplayer
                ovr_series_player_data[player] = ovr_seriesplayer
                ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
                continue
                
            if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                ovrplayer = ovrplayer.add(newplayer)
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                if game == 'Game1':
                    continue
                    
                ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                ovr_series_player_data[player] = ovr_seriesplayer
                ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
            else:
                print(f"{player.title()}'s columns do not match")  
            continue
            
    return new_player_data, ovr_player_data, ovr_series_player_data # Return both dictionaries as a tuple

new_player_data, ovr_player_data, ovr_series_player_data  = load_data(players) # Unpack the tuple of dictionaries into their respective names

No Love file found for Game1.


In [7]:
newButlerStats = PPP(new_player_data['butler'])
newButlerStats.to_csv(f'Butler/PPP Stats/{prevGames[-1] + Opp}.csv')
newButlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.22,34.6,0,11.1,0.29,46.7,14.3,6,16.7,1,0.0,0.0,18.2,0.0,1,0.0,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,0.0,3.8,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,9.1,0.0,0,N/A,1,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.57,26.9,0,57.1,3.0,6.7,100.0,0,N/A,1,100.0,1.33,54.5,50.0,1,100.0,5,40.0
TRANSITION,1.0,7.7,0,50.0,1.0,13.3,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,3.8,1,N/A,0.0,6.7,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [8]:
newButlerSeriesStats = PPP(ovr_series_player_data['butler'])
newButlerSeriesStats.to_csv(f'Butler/PPP Stats/{prevGames[-1] + Opp}.csv')
newButlerSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.22,34.6,0,11.1,0.29,46.7,14.3,6,16.7,1,0.0,0.0,18.2,0.0,1,0.0,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,0.0,3.8,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,9.1,0.0,0,N/A,1,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.57,26.9,0,57.1,3.0,6.7,100.0,0,N/A,1,100.0,1.33,54.5,50.0,1,100.0,5,40.0
TRANSITION,1.0,7.7,0,50.0,1.0,13.3,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,3.8,1,N/A,0.0,6.7,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [9]:
butlerStats = PPP(ovr_player_data['butler'])
butlerStats.to_csv('!PPP Stats/butlerStats.csv')
butlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.96,27.5,6,42.4,0.96,24.8,39.5,33,42.4,5,20.0,0.95,33.5,47.6,10,50.0,11,45.5
PNR Screener,0.0,0.8,0,0.0,0.0,0.6,0.0,1,0.0,0,N/A,0.0,1.3,0.0,0,N/A,1,0.0
DHO Ball Handler,1.5,0.8,0,50.0,1.5,1.1,50.0,1,0.0,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.71,2.7,1,33.3,N/A,0.0,N/A,0,N/A,0,N/A,0.71,8.9,33.3,2,50.0,4,25.0
DBL Ball Handler,0.0,0.8,0,0.0,0.0,0.6,0.0,0,N/A,1,0.0,0.0,1.3,0.0,0,N/A,1,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.07,48.2,7,44.3,0.85,49.7,35.1,61,36.1,13,30.8,1.61,44.9,65.6,10,70.0,22,63.6
TRANSITION,1.64,6.6,2,91.7,1.61,6.2,87.5,8,87.5,0,N/A,1.7,7.4,100.0,2,100.0,2,100.0
Attacking Closeouts,0.67,2.3,2,50.0,0.67,3.3,50.0,4,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.75,1.5,0,100.0,2.75,2.2,100.0,1,100.0,3,100.0,N/A,0.0,N/A,0,N/A,0,N/A


In [10]:
newAdebayoStats = PPP(new_player_data['adebayo'])
newAdebayoStats.to_csv(f'Adebayo/PPP Stats/{prevGames[-1] + Opp}.csv')
newAdebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.8,16.7,2,66.7,0.8,20.8,66.7,3,66.7,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.67,20.0,0,33.3,0.0,16.7,0.0,4,0.0,0,N/A,2.0,33.3,100.0,2,100.0,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.17,40.0,1,63.6,1.2,41.7,60.0,10,60.0,0,N/A,1.0,33.3,100.0,1,100.0,0,N/A
TRANSITION,1.0,6.7,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.0,33.3,50.0,1,100.0,1,0.0
Attacking Closeouts,2.0,3.3,0,100.0,2.0,4.2,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,3.3,0,100.0,2.0,4.2,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [11]:
newAdebayoSeriesStats = PPP(ovr_series_player_data['adebayo'])
newAdebayoSeriesStats.to_csv(f'Adebayo/PPP Stats/{prevGames[-1] + Opp}.csv')
newAdebayoSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.8,16.7,2,66.7,0.8,20.8,66.7,3,66.7,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.67,20.0,0,33.3,0.0,16.7,0.0,4,0.0,0,N/A,2.0,33.3,100.0,2,100.0,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.17,40.0,1,63.6,1.2,41.7,60.0,10,60.0,0,N/A,1.0,33.3,100.0,1,100.0,0,N/A
TRANSITION,1.0,6.7,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.0,33.3,50.0,1,100.0,1,0.0
Attacking Closeouts,2.0,3.3,0,100.0,2.0,4.2,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,3.3,0,100.0,2.0,4.2,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [13]:
adebayoStats = PPP(ovr_player_data['adebayo'])
adebayoStats.to_csv('!PPP Stats/adebayoStats.csv')
adebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.51,2.5,0,50.0,1.51,3.7,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.08,17.2,11,76.2,1.02,20.2,72.2,18,72.2,0,N/A,1.29,11.1,100.0,0,N/A,3,100.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.92,18.3,1,35.5,0.46,7.0,16.7,6,16.7,0,N/A,1.08,41.1,40.0,8,50.0,17,35.3
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.03,44.1,8,52.1,0.91,50.3,45.3,53,45.3,0,N/A,1.4,31.7,72.2,14,78.6,4,50.0
TRANSITION,1.48,7.9,1,69.2,1.33,4.8,80.0,5,80.0,0,N/A,1.58,14.1,62.5,3,100.0,5,40.0
Attacking Closeouts,1.0,1.1,0,50.0,1.0,1.6,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,0.5,0,100.0,2.0,0.8,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [ ]:
newLoveStats = PPP(new_player_data['love'])
newLoveStats.to_csv(f'Love/PPP Stats/{prevGames[-1] + Opp}.csv')
newLoveStats

In [ ]:
loveStats = PPP(ovr_player_data['love'])
loveStats.to_csv('!PPP Stats/loveStats.csv')
loveStats

In [14]:
newLowryStats = PPP(new_player_data['lowry'])
newLowryStats.to_csv(f'Lowry/PPP Stats/{prevGames[-1] + Opp}.csv')
newLowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.38,49.0,0,80.0,2.25,57.1,75.0,1,0.0,3,100.0,2.66,37.6,100.0,0,N/A,1,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.0,16.7,0,50.0,1.0,28.6,50.0,1,100.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,8.3,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,20.0,0.0,0,N/A,1,0.0
TRANSITION,2.0,16.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,40.0,100.0,2,100.0,0,N/A
Attacking Closeouts,0.0,8.3,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,20.0,0.0,0,N/A,1,0.0
Catch & Shoot,0.0,8.3,0,0.0,0.0,14.3,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [15]:
newLowrySeriesStats = PPP(ovr_series_player_data['lowry'])
newLowrySeriesStats.to_csv(f'Lowry/PPP Stats/{prevGames[-1] + Opp}.csv')
newLowrySeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.38,49.0,0,80.0,2.25,57.1,75.0,1,0.0,3,100.0,2.66,37.6,100.0,0,N/A,1,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.0,16.7,0,50.0,1.0,28.6,50.0,1,100.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,8.3,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,20.0,0.0,0,N/A,1,0.0
TRANSITION,2.0,16.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,40.0,100.0,2,100.0,0,N/A
Attacking Closeouts,0.0,8.3,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,20.0,0.0,0,N/A,1,0.0
Catch & Shoot,0.0,8.3,0,0.0,0.0,14.3,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [16]:
lowryStats = PPP(ovr_player_data['lowry'])
lowryStats.to_csv('!PPP Stats/lowryStats.csv')
lowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.92,47.7,8,44.4,0.79,52.7,36.0,15,26.7,10,50.0,1.13,40.9,63.6,8,62.5,3,66.7
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.8,5.2,0,40.0,1.0,7.3,50.0,2,100.0,2,0.0,0.0,2.4,0.0,0,N/A,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.22,11.9,3,75.0,0.86,12.7,75.0,4,75.0,0,N/A,1.8,10.8,75.0,2,100.0,2,50.0
TRANSITION,1.73,18.6,1,75.0,1.29,7.1,33.3,2,0.0,1,100.0,1.86,34.1,84.6,8,87.5,5,80.0
Attacking Closeouts,1.17,6.2,1,60.0,1.33,5.5,66.7,3,66.7,0,N/A,1.0,7.3,50.0,0,N/A,2,50.0
Catch & Shoot,0.86,7.3,0,28.6,0.86,12.7,28.6,0,N/A,7,28.6,N/A,0.0,N/A,0,N/A,0,N/A


In [17]:
newMartinStats = PPP(new_player_data['martin'])
newMartinStats.to_csv(f'Martin/PPP Stats/{prevGames[-1] + Opp}.csv')
newMartinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,22.2,0,0.0,0.0,28.6,0.0,2,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,11.1,0,0.0,0.0,14.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,11.1,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,50.0,0.0,1,0.0,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,11.1,0,0.0,0.0,14.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,22.2,0,0.0,0.0,14.3,0.0,1,0.0,0,N/A,0.0,50.0,0.0,0,N/A,1,0.0
Catch & Shoot,1.5,22.2,0,50.0,1.5,28.6,50.0,0,N/A,2,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [18]:
newMartinSeriesStats = PPP(ovr_series_player_data['martin'])
newMartinSeriesStats.to_csv(f'Martin/PPP Stats/{prevGames[-1] + Opp}.csv')
newMartinSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,22.2,0,0.0,0.0,28.6,0.0,2,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,11.1,0,0.0,0.0,14.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,11.1,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,50.0,0.0,1,0.0,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,11.1,0,0.0,0.0,14.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,22.2,0,0.0,0.0,14.3,0.0,1,0.0,0,N/A,0.0,50.0,0.0,0,N/A,1,0.0
Catch & Shoot,1.5,22.2,0,50.0,1.5,28.6,50.0,0,N/A,2,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [19]:
martinStats = PPP(ovr_player_data['martin'])
martinStats.to_csv('!PPP Stats/martinStats.csv')
martinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.57,6.0,1,33.3,0.57,7.2,33.3,6,33.3,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.67,2.6,0,33.3,0.67,3.1,33.3,1,100.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.5,3.4,0,50.0,1.5,4.1,50.0,2,0.0,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.5,3.4,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.5,21.1,50.0,1,0.0,3,66.7
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.73,18.9,2,36.8,0.85,19.5,38.9,11,63.6,7,0.0,0.0,15.8,0.0,0,N/A,1,0.0
TRANSITION,1.13,13.7,3,66.7,1.09,13.3,60.0,9,66.7,1,0.0,1.33,15.8,100.0,2,100.0,0,N/A
Attacking Closeouts,1.66,17.1,0,73.7,1.6,17.4,75.0,12,83.3,4,50.0,2.0,15.8,66.7,0,N/A,3,66.7
Catch & Shoot,1.97,25.9,0,66.7,1.93,29.9,65.5,1,100.0,28,64.3,3.0,5.3,100.0,0,N/A,1,100.0


In [20]:
newRobinsonStats = PPP(new_player_data['robinson'])
newRobinsonStats.to_csv(f'Robinson/PPP Stats/{prevGames[-1] + Opp}.csv')
newRobinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,9.1,1,N/A,0.0,14.3,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,18.2,0,0.0,0.0,14.3,0.0,0,N/A,1,0.0,0.0,25.0,0.0,0,N/A,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,9.1,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,25.0,0.0,1,0.0,0,N/A
Attacking Closeouts,1.33,27.3,0,66.7,0.0,14.3,0.0,0,N/A,1,0.0,2.0,50.0,100.0,2,100.0,0,N/A
Catch & Shoot,1.0,27.3,0,33.3,1.0,42.9,33.3,0,N/A,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [21]:
newRobinsonSeriesStats = PPP(ovr_series_player_data['robinson'])
newRobinsonSeriesStats.to_csv(f'Robinson/PPP Stats/{prevGames[-1] + Opp}.csv')
newRobinsonSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,9.1,1,N/A,0.0,14.3,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,18.2,0,0.0,0.0,14.3,0.0,0,N/A,1,0.0,0.0,25.0,0.0,0,N/A,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,9.1,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,25.0,0.0,1,0.0,0,N/A
Attacking Closeouts,1.33,27.3,0,66.7,0.0,14.3,0.0,0,N/A,1,0.0,2.0,50.0,100.0,2,100.0,0,N/A
Catch & Shoot,1.0,27.3,0,33.3,1.0,42.9,33.3,0,N/A,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [22]:
robinsonStats = PPP(ovr_player_data['robinson'])
robinsonStats.to_csv('!PPP Stats/robinsonStats.csv')
robinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.5,16.2,1,63.6,1.0,14.5,42.9,2,50.0,5,40.0,2.5,21.1,100.0,2,100.0,2,100.0
PNR Screener,0.0,1.4,0,0.0,0.0,1.8,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.91,26.9,2,41.2,0.93,25.5,35.7,5,40.0,9,33.3,0.85,30.9,66.7,1,100.0,2,50.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.33,8.1,1,20.0,0.33,10.9,20.0,3,33.3,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.67,8.1,1,40.0,0.67,5.5,33.3,1,100.0,2,0.0,0.67,15.8,50.0,2,50.0,0,N/A
Attacking Closeouts,1.22,12.2,1,62.5,1.5,3.6,50.0,0,N/A,2,50.0,1.14,36.8,66.7,5,80.0,1,0.0
Catch & Shoot,1.67,12.2,1,62.5,1.67,16.4,62.5,0,N/A,8,62.5,N/A,0.0,N/A,0,N/A,0,N/A


In [23]:
newStrusStats = PPP(new_player_data['strus'])
newStrusStats.to_csv(f'Strus/PPP Stats/{prevGames[-1] + Opp}.csv')
newStrusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,7.7,0,0.0,0.0,10.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,15.4,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,66.7,0.0,2,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.0,15.4,0,50.0,0.0,10.0,0.0,0,N/A,1,0.0,2.0,33.3,100.0,1,100.0,0,N/A
Catch & Shoot,0.0,53.8,0,0.0,0.0,70.0,0.0,0,N/A,7,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [24]:
newStrusSeriesStats = PPP(ovr_series_player_data['strus'])
newStrusSeriesStats.to_csv(f'Strus/PPP Stats/{prevGames[-1] + Opp}.csv')
newStrusSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,7.7,0,0.0,0.0,10.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,15.4,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,66.7,0.0,2,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.0,15.4,0,50.0,0.0,10.0,0.0,0,N/A,1,0.0,2.0,33.3,100.0,1,100.0,0,N/A
Catch & Shoot,0.0,53.8,0,0.0,0.0,70.0,0.0,0,N/A,7,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [25]:
strusStats = PPP(ovr_player_data['strus'])
strusStats.to_csv('!PPP Stats/strusStats.csv')
strusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.11,23.1,2,50.0,1.14,21.5,50.0,4,50.0,8,50.0,1.0,30.8,50.0,3,66.7,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.58,17.8,0,15.4,0.75,12.3,25.0,0,N/A,8,25.0,0.34,45.2,0.0,4,0.0,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,3.8,0,0.0,0.0,4.6,0.0,1,0.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.05,11.2,0,100.0,2.06,11.9,100.0,5,100.0,1,100.0,2.0,7.7,100.0,1,100.0,0,N/A
Attacking Closeouts,1.16,8.8,1,60.0,1.02,9.0,50.0,3,66.7,1,0.0,2.0,7.7,100.0,1,100.0,0,N/A
Catch & Shoot,0.96,25.5,0,31.6,0.95,29.2,31.6,0,N/A,19,31.6,1.14,6.8,N/A,0,N/A,0,N/A


In [26]:
newVincentStats = PPP(new_player_data['vincent'])
newVincentStats.to_csv(f'Vincent/PPP Stats/{prevGames[-1] + Opp}.csv')
newVincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.5,34.8,0,62.5,1.25,28.6,50.0,2,50.0,2,50.0,1.75,44.4,75.0,2,100.0,2,50.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,13.0,0,0.0,0.0,7.1,0.0,0,N/A,1,0.0,0.0,22.2,0.0,2,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,1.0,8.7,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.0,22.2,50.0,1,100.0,1,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.67,13.0,0,33.3,1.0,14.3,50.0,2,50.0,0,N/A,0.0,11.1,0.0,0,N/A,1,0.0
Catch & Shoot,1.71,30.4,0,57.1,1.71,50.0,57.1,0,N/A,7,57.1,N/A,0.0,N/A,0,N/A,0,N/A


In [27]:
newVincentSeriesStats = PPP(ovr_series_player_data['vincent'])
newVincentSeriesStats.to_csv(f'Vincent/PPP Stats/{prevGames[-1] + Opp}.csv')
newVincentSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.5,34.8,0,62.5,1.25,28.6,50.0,2,50.0,2,50.0,1.75,44.4,75.0,2,100.0,2,50.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,13.0,0,0.0,0.0,7.1,0.0,0,N/A,1,0.0,0.0,22.2,0.0,2,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,1.0,8.7,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.0,22.2,50.0,1,100.0,1,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.67,13.0,0,33.3,1.0,14.3,50.0,2,50.0,0,N/A,0.0,11.1,0.0,0,N/A,1,0.0
Catch & Shoot,1.71,30.4,0,57.1,1.71,50.0,57.1,0,N/A,7,57.1,N/A,0.0,N/A,0,N/A,0,N/A


In [28]:
vincentStats = PPP(ovr_player_data['vincent'])
vincentStats.to_csv('!PPP Stats/vincentStats.csv')
vincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.28,41.6,3,58.5,1.28,42.4,56.2,20,65.0,12,41.7,1.27,39.3,66.7,5,80.0,4,50.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.71,9.1,1,25.0,0.73,8.5,20.0,1,0.0,4,25.0,0.67,10.7,33.3,3,33.3,0,N/A
DHO Screener,3.0,0.9,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,3.6,100.0,0,N/A,1,100.0
DBL Ball Handler,1.19,5.4,0,40.0,1.0,3.7,33.3,2,0.0,1,100.0,1.39,10.3,50.0,1,100.0,1,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.43,10.8,3,14.3,0.51,12.0,14.3,5,20.0,2,0.0,0.0,7.1,N/A,0,N/A,0,N/A
TRANSITION,1.7,5.4,0,60.0,3.0,1.2,100.0,0,N/A,1,100.0,1.43,17.4,50.0,1,100.0,3,33.3
Attacking Closeouts,0.82,9.0,0,25.0,0.77,9.6,16.7,5,20.0,1,0.0,1.0,7.1,50.0,1,100.0,1,0.0
Catch & Shoot,1.76,15.6,0,58.8,1.76,21.0,58.8,0,N/A,17,58.8,N/A,0.0,N/A,0,N/A,0,N/A


In [29]:
newZellerStats = PPP(new_player_data['zeller'])
newZellerStats.to_csv(f'Zeller/PPP Stats/{prevGames[-1] + Opp}.csv')
newZellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,50.0,0,0.0,0.0,100.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [30]:
newZellerSeriesStats = PPP(ovr_series_player_data['zeller'])
newZellerSeriesStats.to_csv(f'Zeller/PPP Stats/{prevGames[-1] + Opp}.csv')
newZellerSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,50.0,0,0.0,0.0,100.0,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [31]:
zellerStats = PPP(ovr_player_data['zeller'])
zellerStats.to_csv('!PPP Stats/zellerStats.csv')
zellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.8,27.8,1,50.0,1.0,16.7,50.0,2,50.0,0,N/A,0.67,50.0,50.0,1,100.0,1,0.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.0,5.6,0,100.0,2.0,8.3,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,4.9,0,N/A,0.0,7.3,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,11.1,0,0.0,0.0,16.7,0.0,2,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [32]:
newTeamStats = PPP(new_player_data['team'])
newTeamStats.to_csv(f'Team/PPP Stats/{prevGames[-1] + Opp}.csv')
newTeamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.0,19.9,1,38.5,0.8,20.8,31.6,13,15.4,6,66.7,1.52,17.9,57.1,3,66.7,4,50.0
PNR Screener,1.02,4.2,2,66.7,1.02,6.1,66.7,3,66.7,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.18,7.9,0,9.1,0.4,5.2,20.0,2,50.0,3,0.0,0.0,13.6,0.0,4,0.0,2,0.0
DHO Screener,0.5,5.7,0,25.0,0.0,4.2,0.0,4,0.0,0,N/A,1.0,9.1,50.0,3,66.7,1,0.0
DBL Ball Handler,0.67,2.1,0,33.3,N/A,0.0,N/A,0,N/A,0,N/A,0.67,6.8,33.3,1,100.0,2,0.0
DBL Screener,2.0,0.7,0,100.0,2.0,1.0,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.19,15.0,1,55.0,1.25,12.5,58.3,11,54.5,1,100.0,1.11,20.5,50.0,2,100.0,6,33.3
TRANSITION,1.27,7.9,1,70.0,1.33,6.2,80.0,5,80.0,0,N/A,1.2,11.4,60.0,4,75.0,1,0.0
Attacking Closeouts,0.67,10.7,1,35.7,0.44,9.4,25.0,5,40.0,3,0.0,1.0,13.6,50.0,3,100.0,3,0.0
Catch & Shoot,1.08,17.1,0,37.5,1.08,25.0,37.5,1,100.0,23,34.8,N/A,0.0,N/A,0,N/A,0,N/A


In [36]:
newOppStats = PPP(new_player_data['opp'])
newOppStats.to_csv(f'Opp/PPP Stats/{prevGames[-1] + Opp}.csv')
newOppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.14,17.1,2,55.0,1.0,15.7,53.8,10,70.0,3,0.0,1.35,19.7,57.1,2,100.0,5,40.0
PNR Screener,1.6,5.1,1,80.0,1.84,5.5,75.0,2,100.0,2,50.0,1.0,4.4,100.0,1,100.0,0,N/A
DHO Ball Handler,1.08,6.2,1,50.0,0.92,6.1,40.0,3,66.7,2,0.0,1.39,6.4,100.0,1,100.0,0,N/A
DHO Screener,0.84,6.2,0,28.6,2.13,2.1,100.0,1,100.0,0,N/A,0.47,14.3,16.7,3,33.3,3,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,2.0,0.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,2.2,100.0,1,100.0,0,N/A
ISOLATION,0.98,24.4,4,46.2,0.75,22.3,41.2,15,46.7,2,0.0,1.33,28.4,55.6,2,100.0,7,42.9
TRANSITION,1.06,11.3,1,36.4,0.98,10.3,33.3,5,40.0,1,0.0,1.19,13.1,40.0,2,50.0,3,33.3
Attacking Closeouts,1.56,4.3,0,75.0,2.08,3.2,100.0,1,100.0,1,100.0,1.04,6.4,50.0,1,100.0,1,0.0
Catch & Shoot,1.2,11.2,0,40.0,1.2,16.9,40.0,0,N/A,15,40.0,N/A,0.0,N/A,0,N/A,0,N/A


In [33]:
TeamSeriesStats = PPP(ovr_series_player_data['team'])
TeamSeriesStats.to_csv(f'Team/PPP Stats/{prevGames[-1] + Opp}.csv')
TeamSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.0,19.9,1,38.5,0.8,20.8,31.6,13,15.4,6,66.7,1.52,17.9,57.1,3,66.7,4,50.0
PNR Screener,1.02,4.2,2,66.7,1.02,6.1,66.7,3,66.7,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.18,7.9,0,9.1,0.4,5.2,20.0,2,50.0,3,0.0,0.0,13.6,0.0,4,0.0,2,0.0
DHO Screener,0.5,5.7,0,25.0,0.0,4.2,0.0,4,0.0,0,N/A,1.0,9.1,50.0,3,66.7,1,0.0
DBL Ball Handler,0.67,2.1,0,33.3,N/A,0.0,N/A,0,N/A,0,N/A,0.67,6.8,33.3,1,100.0,2,0.0
DBL Screener,2.0,0.7,0,100.0,2.0,1.0,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.19,15.0,1,55.0,1.25,12.5,58.3,11,54.5,1,100.0,1.11,20.5,50.0,2,100.0,6,33.3
TRANSITION,1.27,7.9,1,70.0,1.33,6.2,80.0,5,80.0,0,N/A,1.2,11.4,60.0,4,75.0,1,0.0
Attacking Closeouts,0.67,10.7,1,35.7,0.44,9.4,25.0,5,40.0,3,0.0,1.0,13.6,50.0,3,100.0,3,0.0
Catch & Shoot,1.08,17.1,0,37.5,1.08,25.0,37.5,1,100.0,23,34.8,N/A,0.0,N/A,0,N/A,0,N/A


In [37]:
OppSeriesStats = PPP(ovr_series_player_data['opp'])
OppSeriesStats.to_csv(f'Opp/PPP Stats/{prevGames[-1] + Opp}.csv')
OppSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.14,17.1,2,55.0,1.0,15.7,53.8,10,70.0,3,0.0,1.35,19.7,57.1,2,100.0,5,40.0
PNR Screener,1.6,5.1,1,80.0,1.84,5.5,75.0,2,100.0,2,50.0,1.0,4.4,100.0,1,100.0,0,N/A
DHO Ball Handler,1.08,6.2,1,50.0,0.92,6.1,40.0,3,66.7,2,0.0,1.39,6.4,100.0,1,100.0,0,N/A
DHO Screener,0.84,6.2,0,28.6,2.13,2.1,100.0,1,100.0,0,N/A,0.47,14.3,16.7,3,33.3,3,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,2.0,0.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,2.2,100.0,1,100.0,0,N/A
ISOLATION,0.98,24.4,4,46.2,0.75,22.3,41.2,15,46.7,2,0.0,1.33,28.4,55.6,2,100.0,7,42.9
TRANSITION,1.06,11.3,1,36.4,0.98,10.3,33.3,5,40.0,1,0.0,1.19,13.1,40.0,2,50.0,3,33.3
Attacking Closeouts,1.56,4.3,0,75.0,2.08,3.2,100.0,1,100.0,1,100.0,1.04,6.4,50.0,1,100.0,1,0.0
Catch & Shoot,1.2,11.2,0,40.0,1.2,16.9,40.0,0,N/A,15,40.0,N/A,0.0,N/A,0,N/A,0,N/A


In [34]:
teamStats = PPP(ovr_player_data['team'])
teamStats.to_csv('!PPP Stats/teamStats.csv')
teamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.06,20.5,21,48.3,1.02,19.8,43.9,83,44.6,40,42.5,1.16,22.1,59.2,28,64.3,21,52.4
PNR Screener,0.97,4.7,13,60.6,0.99,4.9,59.3,23,65.2,4,25.0,0.92,4.3,66.7,1,100.0,5,60.0
DHO Ball Handler,0.81,5.7,4,32.0,0.95,5.4,35.1,11,36.4,26,34.6,0.51,6.3,23.1,8,25.0,5,20.0
DHO Screener,0.95,4.9,2,36.4,0.41,1.4,14.3,6,16.7,1,0.0,1.08,13.8,40.5,11,45.5,26,38.5
DBL Ball Handler,0.89,0.8,0,28.6,0.75,0.6,25.0,2,0.0,2,50.0,1.03,1.4,33.3,1,100.0,2,0.0
DBL Screener,2.0,0.1,0,100.0,2.0,0.1,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.97,27.2,24,45.0,0.82,28.7,37.6,147,41.5,26,15.4,1.44,23.4,67.9,27,77.8,29,58.6
TRANSITION,1.52,9.8,9,75.0,1.45,7.9,73.3,35,80.0,10,50.0,1.61,14.8,77.1,20,90.0,15,60.0
Attacking Closeouts,1.17,6.3,5,55.6,1.12,6.5,52.5,31,58.1,9,33.3,1.31,5.7,64.3,7,85.7,7,42.9
Catch & Shoot,1.67,10.1,2,58.2,1.64,13.6,57.3,4,100.0,92,55.4,2.43,1.0,100.0,0,N/A,2,100.0


In [38]:
oppStats = PPP(ovr_player_data['opp'])
oppStats.to_csv('!PPP Stats/oppStats.csv')
oppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.96,14.1,19,44.6,0.77,11.2,34.8,44,36.4,22,31.8,1.19,20.9,56.4,25,76.0,30,40.0
PNR Screener,1.31,5.2,7,61.9,1.41,5.1,67.7,20,80.0,11,45.5,1.07,5.2,45.5,1,0.0,10,50.0
DHO Ball Handler,0.84,1.8,0,31.6,1.1,1.4,40.0,2,50.0,8,37.5,0.56,2.8,22.2,5,20.0,4,25.0
DHO Screener,0.69,1.2,1,33.3,0.0,0.7,0.0,4,0.0,1,0.0,1.12,2.5,57.1,4,75.0,3,33.3
DBL Ball Handler,1.22,0.9,0,55.6,0.67,0.4,33.3,3,33.3,0,N/A,1.5,1.9,66.7,3,100.0,3,33.3
DBL Screener,2.25,0.4,0,100.0,2.25,0.5,100.0,3,100.0,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.99,20.9,21,46.7,0.98,20.6,46.3,92,54.3,31,22.6,1.02,21.4,47.4,24,70.8,33,30.3
TRANSITION,1.13,14.8,18,54.2,1.12,12.3,55.1,53,64.2,16,25.0,1.13,20.4,53.1,24,83.3,25,24.0
Attacking Closeouts,0.96,13.0,12,41.8,0.88,12.7,40.0,52,48.1,18,16.7,1.13,13.5,45.0,10,80.0,30,33.3
Catch & Shoot,0.94,16.9,3,32.2,0.95,23.8,32.2,2,100.0,169,31.4,0.5,1.2,33.3,1,100.0,2,0.0
